In [ ]:
!git clone https://github.com/LINKS-Foundation-CPE/Master-QCC-2022-2023.git

In [ ]:
!pip install pulser==0.7
!pip install pybnb
!pip install networkx==2.8.7
!pip install matplotlib==3.5.1

In [ ]:
%cd /content/Master-QCC-2022-2023/

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from pulser import Pulse, Sequence, Register
from pulser.devices import Chadoq2
from pulser_simulation import SimConfig, Simulation

from scipy.optimize import minimize, Bounds
from src.basic_MIS.utils import *

In [ ]:
# pos = np.array([[0., 0.], [-4, -7], [4,-7], [8,6], [-8,6]])
pos = np.array([[0., 0.], [-4, -7], [4,-7], [8,6], [-8,6], [8,16], [-8,16], [0,20]])
# pos = np.array([[0., 0.], [-4, -7], [4,-7], [8,6], [-8,6], [8,16], [-8,16], [0,20], [16,18], [-18,12]])

G = pos_to_graph(pos)
qubits = dict(enumerate(pos))

reg = Register(qubits)
reg.draw(blockade_radius=Chadoq2.rydberg_blockade_radius(1.), draw_graph=True, draw_half_radius=True ) #Assuming Rabi frequency = 1. rad/microsec

mis_ref = nx_mis(G)
print(mis_ref)

In [ ]:
layers = 1
time_unit = 1000 # = 1 microsecond
detuning_amplitude = 1. # Must be between 0 and Rabi frequency, 1. is reasonable
guess = {'t': np.random.uniform(8, 10, layers),
         's': np.random.uniform(1, 3, layers)}

# Parametrized sequence
seq = Sequence(reg, Chadoq2)
seq.declare_channel('ch0','rydberg_global')

t_list = seq.declare_variable('t_list', size=layers)
s_list = seq.declare_variable('s_list', size=layers)

if layers == 1:
    t_list = [t_list]
    s_list = [s_list]

for t, s in zip(t_list, s_list):
    pulse_1 = Pulse.ConstantPulse(time_unit*t, 1., 0., 0)
    pulse_2 = Pulse.ConstantPulse(time_unit*s, 1., detuning_amplitude, 0)

    seq.add(pulse_1, 'ch0')
    seq.add(pulse_2, 'ch0')

# Bounds for max total pulse length (machine max = 100, reasonable values < 20)
max_time = 10.
step_time = max_time/layers
lb = []
ub = []
for t in t_list:
    lb.append(0.016) # minimum length of laser pulse
    ub.append(step_time*0.7)
for s in s_list:
    lb.append(0.016)
    ub.append(step_time*0.3)
np_ub = np.array(ub)
np_lb = np.array(lb)

In [ ]:
def quantum_loop(parameters, draw=False, printx=False):
    if printx==True:
        print(parameters)
    t_params, s_params = 4 * (np.rint( (np.reshape(parameters, (2, layers)) * 1000) /4 ) / 1000 ) # Ensure value is multiple of 4ns
    assigned_sequence = seq.build(t_list=t_params, s_list=s_params)
    assigned_sequence.measure('ground-rydberg')
    if draw:
        assigned_sequence.draw()
    
    simul = Simulation(assigned_sequence, sampling_rate=0.1)
    results = simul.run(nsteps=1e10)
    return results.sample_final_state()

In [ ]:
example_dict = quantum_loop(np.r_[guess['t'], guess['s']], draw=True)
plot_distribution(example_dict, mis_ref)

In [ ]:
# MIS cost function
def get_cost_MIS(bitstring, G, penalty=1):
    z = np.array(list(bitstring), dtype=int)
    A = nx.to_numpy_matrix(G)
    # Add penalty and bias:
    cost = penalty*(z.T @ np.triu(A) @ z) - np.sum(z)
    return cost

# Cost of whole simulation with given params (averages MIS cost over many samples)
def get_cost(counter,G):
    cost = sum(counter[key] * get_cost_MIS(key,G) for key in counter)
    return cost / sum(counter.values()) # Divide by total samples

# Run simulation and get cost
def func(param,*args):
    G = args[0]
    C = quantum_loop(param);
    cost = get_cost(C,G)
    return cost

print(get_cost_MIS(list(mis_ref)[0], G)) # Cost of reference solution
print(get_cost(example_dict, G)) # Cost of examples solution

In [ ]:
res = minimize(func,
               args=G,
               x0=np.array([1. for i in range(2*layers)]),
               method='Nelder-Mead',
               tol=1e-6,
            #    bounds=Bounds(np_lb,np_ub),
               options = {'maxiter': 50, 'disp': True}
              )

print(res.x)
count_dict = quantum_loop(res.x, draw=True)
print(get_cost(count_dict, G))
plot_distribution(count_dict, mis_ref)

In [ ]:
cmap = ['royalblue','coral']
clist = [cmap[int(c)] for c in get_mis_string(count_dict)]
f = plt.figure()
nx.draw(G, pos=pos, node_color=clist, with_labels=True, ax=f.add_subplot(111))